# Run the ReAction agent in batch

Run for all:
- Tasks
- Models

In [ ]:
from pathlib import Path
from datetime import date, timedelta
import time

import papermill as pm

from react_repl_agent.utils.timedelta import format_timedelta

## Setup notebook to run and export directory

In [ ]:
notebook_to_run_path = Path('./react_repl_agent.ipynb').resolve()
assert notebook_to_run_path.exists()
print(notebook_to_run_path)

In [ ]:
notebooks_export_dir = Path(
    "./export/papermill_notebooks_export_"+ date.today().isoformat()
).expanduser()
notebooks_export_dir.mkdir(parents=True, exist_ok=True)

## Define batch parameters

In [ ]:
models = ["gpt-3.5-turbo", "text-davinci-003", "gpt-4"]
tasks = [
    {
        "name": "A_print_date",
        "description": "Print the current date.",
    },
    {
        "name": "B_Weather_MyLocation_Discord",
        "description": "Send a Discord message with the current weather for my location.",
    },
    {
        "name": "C_Weather_London_Slack",
        "description": "Send a Slack message to the #experimenting-with-simple-ai-agents channel with the current weather for London, UK.",
    },
    {
        "name": "D_Weather_Emoji_MyLocation_Discord",
        "description": "Send a descriptive Discord message with the weather forecast for the next 5 days for my location. Describe the weather each day with an appropriate emoji, include the dates in the message.",
    },
    {
        "name": "E_NASA_Discord",
        "description": "Lookup the NASA picture-of-the day, resize to half it's size, and send to discord with a message starting with 'Hello from Space!' and providing additional information about the image.",
    },
    {
        "name": "F_MSFT_Stock_Slack",
        "description": "What week in the last 3 months had the lowest MSFT stock closing price? Send the result including price and date to the #experimenting-with-simple-ai-agents Slack channel.",
    },
]

## Run batch

In [ ]:
for model in models:
    for task in tasks:
        print(f"\n# Running '{model}' for '{task['name']}'")
        notebook_export_path = notebooks_export_dir / model / f"{model}_{task['name']}.ipynb"
        notebook_export_path = notebook_export_path.resolve()
        notebook_export_path.parent.mkdir(parents=True, exist_ok=True)
        print(f"Exporting to '{notebook_export_path!s}'.")
        try:
            start_time = time.time()
            pm.execute_notebook(
                notebook_to_run_path,
                notebook_export_path,
                parameters=dict(
                    llm_model=model,
                    task_name=task["name"],
                    task_description=task["description"],
                ),
            )
            runtime = timedelta(seconds=time.time() - start_time)
            print(f"SUCCESS in {format_timedelta(runtime)}!")
        except pm.PapermillExecutionError as exc:
            print(f"ERROR!  {exc.ename}: {exc.evalue}")
print("DONE!")